<a href="https://colab.research.google.com/github/JohnnySolo/Data-Analysis-Project---Feedback-Study/blob/main/Data_Analysis_Project_Feedback_Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Statistical Models B - EX07 (LMM & GLRT)

In [61]:
# Step 1: Create SSH directory and known_hosts file
!mkdir -p ~/.ssh
!touch ~/.ssh/known_hosts

# Step 2: Add GitHub to known_hosts
!ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts

# Step 3: Start the SSH agent and add the private key
!eval $(ssh-agent -s)
!ssh-add ~/.ssh/id_rsa

# Step 4: Test the SSH connection
!ssh -i ~/.ssh/id_rsa -T git@github.com

# Step 5: Clone the repository
!git clone git@github.com:JohnnySolo/Data-Analysis-Project---Feedback-Response.git

!ssh -T git@github.com

# github.com:22 SSH-2.0-159e461a3
Agent pid 31501
Could not open a connection to your authentication agent.
Hi JohnnySolo! You've successfully authenticated, but GitHub does not provide shell access.
fatal: destination path 'Data-Analysis-Project---Feedback-Response' already exists and is not an empty directory.
Hi JohnnySolo! You've successfully authenticated, but GitHub does not provide shell access.


In [94]:
# Automate pushing Colab notebook updates to GitHub
import os

# Define paths and repository details
repo_dir = "/content/Data-Analysis-Project---Feedback-Study"
notebook_path = "/content/YourNotebookName.ipynb"  # Replace with your notebook's actual name
repo_url = "git@github.com:JohnnySolo/Data-Analysis-Project---Feedback-Study.git"

# Clone the repository if it doesn't exist locally
if not os.path.exists(repo_dir):
    !git clone {repo_url} {repo_dir}

# Move to the repository directory
%cd {repo_dir}

# Move the notebook to the repository
!cp {notebook_path} {repo_dir}/

# Add changes to Git
!git add .

# Commit changes with a message
!git commit -m "Update Colab notebook for Feedback Study project"

# Push changes to GitHub
!git push -u origin main

/content/Data-Analysis-Project---Feedback-Study
cp: cannot stat '/content/YourNotebookName.ipynb': No such file or directory
error: 'Data-Analysis-Project---Feedback-Response/' does not have a commit checked out
fatal: adding files failed
On branch main

Initial commit

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Data-Analysis-Project---Feedback-Response/

nothing added to commit but untracked files present (use "git add" to track)
error: src refspec main does not match any
error: failed to push some refs to 'github.com:JohnnySolo/Data-Analysis-Project---Feedback-Study.git'


In [ ]:
# Load necessary libraries
import pandas as pd  # For handling data frames and CSV file operations
import statsmodels.formula.api as smf  # For specifying statistical models using formulas
from statsmodels.regression.mixed_linear_model import MixedLM  # For fitting mixed linear models
import statsmodels.api as sm  # For performing advanced statistical tests
from scipy.stats import chi2  # For chi-squared distribution calculations
import numpy as np  # For matrix and vector computations

In [ ]:
# Upload and Read the CSV file
from google.colab import files  # For uploading files in Google Colab
uploaded = files.upload()  # Prompt to upload the file

# Update the file path to match the uploaded file name
feedback_df = pd.read_csv("feedback_df_bi.csv")  # Use the correct file name after upload


Saving feedback_df_bi.csv to feedback_df_bi.csv


### **Feedback Linear Mixed Model (LMM)**

We model the performance score outcome \\( [\text{performance}_{ij}] \\) of individuals as a function of their gender \\( [\text{gender}_{ij}] \\), the feedback they got \\( [\text{feedback}_{ij}] \\), and their interaction \\( [\text{gender}_{ij}] \cdot [\text{feedback}_{ij}] \\), while using \\( [\text{id}_{i}] \\) as a cluster.  

Therefore, we'll be using a Linear Mixed Model (LMM).  

The full formulation of the model is:

\\[ \text{performance}_{ij} = \begin{bmatrix} 1 & \text{gender}_{ij} & \text{feedback}_{ij} & \text{gender}_{ij} \cdot \text{feedback}_{ij} \end{bmatrix} \cdot \begin{bmatrix} \beta_0 \\ \beta_1 \\ \beta_2 \\ \beta_3 \end{bmatrix} + \text{id}_i \cdot b_{0i} + \epsilon_{ij} \\]

#### <u>Where</u>:

- **\\( \text{performance}_{ij} \\)**:  
  Outcome of the performance vector (observation performance score).

- **\\( \beta_0 \\)**:  
  The overall intercept - baseline performance score when all predictors are zero (**fixed effect**).  

- **\\( \text{gender}_{ij} \\)**:  
  The gender predictor for observation \\( j \\) in cluster \\( i \\).  

- **\\( \beta_1 \\)**:  
  The effect of \\( \text{gender}_{ij} \\) on \\( \text{performance}_{ij} \\) (**fixed effect**).  

- **\\( \text{feedback}_{ij} \\)**:  
  The feedback type predictor for observation \\( j \\) in cluster \\( i \\).  

- **\\( \beta_2 \\)**:  
  The effect of \\( \text{feedback}_{ij} \\) type on \\( \text{performance}_{ij} \\) (**fixed effect**).  

- **\\( \text{gender}_{ij} \cdot \text{feedback}_{ij} \\)**:  
  The interaction term between \\( \text{gender}_{ij} \\) and \\( \text{feedback}_{ij} \\).  

- **\\( \beta_3 \\)**:  
  The interaction effect between \\( \text{gender}_{ij} \\) and \\( \text{feedback}_{ij} \\) (**fixed effect**).  

- **\\( \text{id}_{i} \\)**:  
  The \\( \text{id}_{i} \\) cluster predictor.  

- **\\( b_{0i} \\)**:  
  Cluster intercept (**random effect**).  

- **\\( \epsilon_{ij} \\)**:  
  Vector of residual errors.  



#### <u>The model assumptions are</u>:

- \\( \boldsymbol{b}_{0i} \sim N(\mathbf{0}, \sigma^2_{b_0}) \\).  

- \\( \boldsymbol{\epsilon} \sim N(\mathbf{0}, \sigma^2_\epsilon{I}) \\).  

- \\( \boldsymbol{b}_{0i} \\) and \\( \boldsymbol{\epsilon} \\) are \\( iid \\).  


### Generalized Likelihood Ratio Test (GLRT)

Based on what we study, the **Generalized Likelihood Ratio Test (GLRT)** is a statistical test used to compare the goodness-of-fit of two nested models:
- A **full model** (more complex, with additional parameters).
- A **reduced model** (simpler, with fewer parameters).

The test evaluates whether the additional parameters in the full model significantly improve the model fit. This is particularly useful in mixed models, such as testing the significance of random slopes or interaction effects.

---

#### **Test Statistic**

The GLRT is based on the likelihood ratio:

\\[ \Lambda = -2 \cdot \left[ \log L(\text{reduced model}) - \log L(\text{full model}) \right] \\]

Where:
- \\( L(\cdot) \\): The likelihood of the model.

Test Assumptions:

1. **The likelihood ratio test statistic approximately follows a chi-squared distribution:**
   The test statistic \\( \Lambda \\) follows a chi-squared distribution under the null hypothesis, with degrees of freedom equal to the difference in the number of parameters between the models. That's what allows us to do Hypothesis test on the full model parameters.
2. **Models are nested:**
   The reduced model is a special case of the full model. Mathematically:
   
   \\(\text{Reduced Model: } Y_{ij} = f(X_{ij}; \beta_{\text{reduced}})\\)
   
   \\(\text{Full Model: } Y_{ij} = f(X_{ij}; \beta_{\text{full}})\\)
   
   Where:
   - \\( \beta_{\text{reduced}} \subseteq \beta_{\text{full}} \\), meaning the reduced model's parameters are a subset of the full model's parameters.

---

#### **Hypotheses**

1. **Null Hypothesis $( H_0 )$:**
   - The additional parameters in the full model are **not significant**.

2. **Alternative Hypothesis $( H_1 )$:**
   - The additional parameters in the full model are **significant** and improve the model fit.

---

##### a. <u> Test Random Effects </u>

We compare:
- **Full model:** Random intercept and random slope.    
- **Reduced model:** Random intercept only.  

The null hypothesis $( H_0 )$ is:   

\\[\boldsymbol{b}_{0i} = \mathbf{0} \\]   

The alternative hypothesis $( H_1 )$ is:   

\\[ \boldsymbol{b}_{0i} > \mathbf{0} \\]  

---

We'll perform GLRT manually, based on this theorem. We'll compare the two models and provide the \\( \chi^2 \\) test statistic and \\( \boldsymbol{p\text{-}value} \\) accordingly.
If the \\( \boldsymbol{p\text{-}value} \\) for the likelihood ratio test is significant, the random intercept is meaningful.

In [ ]:
# Step 1: Defining the Linear Mixed Model (LMM)
# Formula explanation:
# 'performance ~ gender + feedback + gender:feedback' specifies the fixed effects.
# 'id' is included as a grouping factor for random effects.

model_formula = "performance ~ gender + feedback + gender:feedback"

# Step 2: Fitting the Linear Mixed Model
# Using statsmodels MixedLM to fit the model with random intercept by 'id'
lmm = MixedLM.from_formula(model_formula, groups=feedback_df['id'], data=feedback_df)
lmm_result = lmm.fit()

# Display the summary of the model
print(lmm_result.summary())

# Step 3: Comparing Full and Reduced Models for GLRT
# Full model: With Random intercept
full_model = MixedLM.from_formula("performance ~ gender * feedback", groups=feedback_df["id"], data=feedback_df).fit()

# Reduced model: Without Random intercept
reduced_model = smf.ols("performance ~ gender * feedback", data=feedback_df).fit()

# Perform GLRT manually
log_likelihood_full = full_model.llf  # Log-likelihood of the full model
log_likelihood_reduced = reduced_model.llf  # Log-likelihood of the reduced model

# Calculate the test statistic
likelihood_ratio_statistic = -2 * (log_likelihood_reduced - log_likelihood_full)

# Degrees of freedom
df_diff = full_model.df_modelwc - reduced_model.df_model  # Difference in the number of parameters

# Calculate the p-value using the chi-squared distribution
p_value = chi2.sf(likelihood_ratio_statistic, df_diff)

print(f"Likelihood Ratio Statistic: {likelihood_ratio_statistic}")
print(f"Degrees of Freedom: {df_diff}")
print(f"P-Value: {p_value}")

                   Mixed Linear Model Regression Results
Model:                   MixedLM       Dependent Variable:       performance
No. Observations:        792           Method:                   REML       
No. Groups:              22            Scale:                    119.6287   
Min. group size:         36            Log-Likelihood:           -3042.9947 
Max. group size:         36            Converged:                Yes        
Mean group size:         36.0                                               
----------------------------------------------------------------------------
                                Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
----------------------------------------------------------------------------
Intercept                      105.128    6.568 16.006 0.000  92.255 118.002
feedback[T.no feedback]        -17.192    3.010 -5.711 0.000 -23.093 -11.292
feedback[T.positive]           -18.638    3.010 -6.191 0.000 -24.538 -12.738
gender             

**Final Answer:**  
For $\alpha = 0.05$, We'll reject $H_0$.  
We conclude that the random intercept \\( \boldsymbol{b}_{0i} \neq \mathbf{0} \\) and it's significant.  
Meaning the \\( \mathbf{id}_{i} \\) has an effect on the performance variability.

##### a. <u> Test Interaction Effects </u>  

Now we'll test if the interaction is significant.    

We compare:  
- **Full model:** With interaction  
- **Reduced model:** Without interaction.  

The null hypothesis ($H_0$) is:  

\\[ \boldsymbol{\beta}_{3} = \mathbf{0} \\]    

The alternative hypothesis ($H_1$) is:

\\[ \boldsymbol{\beta}_{3} > \mathbf{0} \\]   

We'll use the same method from earlier to perform GLRT.

In [ ]:
# Step 1: Defining the Linear Mixed Model (LMM)
# Formula explanation:
# 'performance ~ gender + feedback + gender:feedback' specifies the fixed effects.
# 'id' is included as a grouping factor for random effects.

model_formula = "performance ~ gender + feedback + gender:feedback"

# Step 2: Fitting the Linear Mixed Model
# Using statsmodels MixedLM to fit the model with random intercept by 'id'
lmm = MixedLM.from_formula(model_formula, groups=feedback_df['id'], data=feedback_df)
lmm_result = lmm.fit()

# Display the summary of the model
print(lmm_result.summary())

# Step 3: Comparing Full and Reduced Models for GLRT
# Full model: With interaction (random intercept included)
full_model = MixedLM.from_formula("performance ~ gender * feedback", groups=feedback_df["id"], data=feedback_df).fit()

# Reduced model: Without interaction (random intercept included)
reduced_model = MixedLM.from_formula("performance ~ gender + feedback", groups=feedback_df["id"], data=feedback_df).fit()

# Perform GLRT manually
log_likelihood_full = full_model.llf  # Log-likelihood of the full model
log_likelihood_reduced = reduced_model.llf  # Log-likelihood of the reduced model

# Calculate the test statistic
likelihood_ratio_statistic = -2 * (log_likelihood_reduced - log_likelihood_full)

# Degrees of freedom
df_diff = full_model.df_modelwc - reduced_model.df_modelwc  # Difference in the number of parameters

# Calculate the p-value using the chi-squared distribution
p_value = chi2.sf(likelihood_ratio_statistic, df_diff)

print(f"Likelihood Ratio Statistic: {likelihood_ratio_statistic}")
print(f"Degrees of Freedom: {df_diff}")
print(f"P-Value: {p_value}")

                   Mixed Linear Model Regression Results
Model:                   MixedLM       Dependent Variable:       performance
No. Observations:        792           Method:                   REML       
No. Groups:              22            Scale:                    119.6287   
Min. group size:         36            Log-Likelihood:           -3042.9947 
Max. group size:         36            Converged:                Yes        
Mean group size:         36.0                                               
----------------------------------------------------------------------------
                                Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
----------------------------------------------------------------------------
Intercept                      105.128    6.568 16.006 0.000  92.255 118.002
feedback[T.no feedback]        -17.192    3.010 -5.711 0.000 -23.093 -11.292
feedback[T.positive]           -18.638    3.010 -6.191 0.000 -24.538 -12.738
gender             

**Final Answer:**  
For $\alpha = 0.05$, We'll reject $H_0$.  
We conclude that the interaction term \\( \boldsymbol{\beta}_{3} \neq \mathbf{0} \\) and it's significant.  
Meaning we can assume that there's a dependency between \\( \text{gender}_{ij} \\) and \\( \text{feedback}_{ij} \\).

#### b. <u> Bayesian Estimation </u>

It can be shown that under our model

\\[ \boldsymbol{b}_{i} | \boldsymbol{Y}_{i}, \boldsymbol{X}_{i}, \boldsymbol{Z}_{i} \sim N(\mu_{b_i}, \Sigma_{b_i}).\\]

Where:  

- \\( \mu_{b_i} = \boldsymbol{B}\boldsymbol{Z}_{i}^T \Sigma_{i}^{-1} (\boldsymbol{Y}_{i} - \boldsymbol{X}_{i}\boldsymbol{\beta}) \\)  

- \\( \Sigma_{b_i} = \boldsymbol{B} - \boldsymbol{B}\boldsymbol{Z}_{i}^T \Sigma_{i}^{-1} \boldsymbol{Z}_{i} \boldsymbol{B} \\)

Moreover, Based on our model assumptions:

\\[ \boldsymbol{Y}_{i} = \boldsymbol{X}_{i} \cdot \boldsymbol{\beta} + \boldsymbol{Z}_{i} \cdot \boldsymbol{b}_{i} + \boldsymbol{\epsilon}_{ij} \\]

Where:

\\[ \boldsymbol{Y}_{i} \sim N(\boldsymbol{X}_{i} \cdot \boldsymbol{\beta} , \Sigma_{i}) \\]

**Note**: \\( \Sigma_{i} = \boldsymbol{Z}_{i} \cdot \boldsymbol{B} \cdot \boldsymbol{Z^T}_{i} + \sigma^2_\epsilon{I} \\)

We'll be focusing on specific clusters (\\( \text{id}_{7} \\) and \\( \text{id}_{12} \\)).  
we'll calculate \\( \hat{b}_i \\) based on the formula and then compare it to results from a designated function.

##### <u> Calculation based on formula</u>

In [ ]:
# Define the function to calculate b_i_hat for a given cluster ID
def find_feedback_bi(i):
    # Create B (variance of random intercept)
    B = lmm_result.cov_re.iloc[0, 0]  # Extract random intercept variance

    # Create Z_i (design matrix for random effects, in this case, just ones)
    rows_cluster = feedback_df[feedback_df['id'] == i].index  # Identify row indexes for cluster i
    Z_i = np.ones((len(rows_cluster), 1))  # Vector of 1's

    # Subset all the outcomes (performance) for cluster i
    Y_i = feedback_df.loc[rows_cluster, 'performance']

    # Create X_i (design matrix for fixed effects)
    X_full = lmm_result.model.exog  # Extract full design matrix
    X_i = X_full[rows_cluster, :]  # Subset design matrix for cluster i

    # Fixed-effects coefficients (beta)
    beta = lmm_result.fe_params.values  # Extract fixed-effects coefficients

    # Compute Sigma_i (residual covariance matrix)
    residual_var = lmm_result.scale  # Residual variance
    Sigma_i_inv = np.linalg.inv(Z_i @ Z_i.T * B + residual_var * np.eye(len(rows_cluster)))

    # Compute b_i_hat
    b_i_hat = B * Z_i.T @ Sigma_i_inv @ (Y_i.values - X_i @ beta)

    return b_i_hat

# Calculate b_i_hat for clusters 7 and 12
print("Our beta estimations for id=7 and id=12 are:", find_feedback_bi(7), "and", find_feedback_bi(12), "accordingly")


Our beta estimations for id=7 and id=12 are: [6.12546163] and [-9.22028355] accordingly


##### Calculation based on designated ```random_effects``` function from ```statmodels```

In [ ]:
# Extract the random effects for all clusters
random_effects = lmm_result.random_effects

# Print the random effects for specific clusters (e.g., 7 and 12)
print("Random effect for id=7:", random_effects[7])
print("Random effect for id=12:", random_effects[12])


Random effect for id=7: Group    6.125462
dtype: float64
Random effect for id=12: Group   -9.220284
dtype: float64


We can see that the results are the same, so we didn't had to work hard in the first place 🫠

In [87]:
!pwd
!ls


/content/Data-Analysis-Project---Feedback-Study
Data-Analysis-Project---Feedback-Response


In [84]:
!rm -rf /content/Feedback-Response-Project
!rm -rf /content/Data-Analysis-Project---Feedback-Response



In [85]:
%cd /content
!git clone git@github.com:JohnnySolo/Data-Analysis-Project---Feedback-Study.git
%cd Data-Analysis-Project---Feedback-Study


/content
fatal: destination path 'Data-Analysis-Project---Feedback-Study' already exists and is not an empty directory.
/content/Data-Analysis-Project---Feedback-Study


In [86]:
!ls /content/


Data-Analysis-Project---Feedback-Study	individual-Data-Analysis-Projects  sample_data


In [89]:
!mv /content/Data-Analysis-Project---Feedback-Study.ipynb /content/Data-Analysis-Project---Feedback-Study/


mv: cannot stat '/content/Data-Analysis-Project---Feedback-Study.ipynb': No such file or directory


In [90]:
!ls /content/Data-Analysis-Project---Feedback-Study/


Data-Analysis-Project---Feedback-Response


In [91]:
%cd /content/Data-Analysis-Project---Feedback-Study
!git remote -v


/content/Data-Analysis-Project---Feedback-Study
origin	git@github.com:JohnnySolo/Data-Analysis-Project---Feedback-Study.git (fetch)
origin	git@github.com:JohnnySolo/Data-Analysis-Project---Feedback-Study.git (push)


In [92]:
!git add .
!git commit -m "Added Colab notebook for Feedback Study project"


error: 'Data-Analysis-Project---Feedback-Response/' does not have a commit checked out
fatal: adding files failed
On branch main

Initial commit

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Data-Analysis-Project---Feedback-Response/

nothing added to commit but untracked files present (use "git add" to track)


In [93]:
!git push -u origin main


error: src refspec main does not match any
error: failed to push some refs to 'github.com:JohnnySolo/Data-Analysis-Project---Feedback-Study.git'
